In [ ]:
import sys
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.cross_decomposition import CCA
import numpy as np
import pandas as pd
import json
from sklearn.decomposition import PCA

In [ ]:
# Read in the list of qustions from the file
qs_file = "File with list of default questions"
base_qs = pd.read_csv(qs_file)

In [114]:
def get_app_qs(filename):
    # Get file with curated questions
    # This is specific to the type of file being used
    # i.e. can add code here are needed
    # Returns a dataframe

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

In [115]:
def get_cca(veca, vecb):
    """ USE CCA to get the correlation for the first canonical pair"""
    # Get the first canonical correlation
    cca_vals = []
    for qb in vecb:
        cca = CCA(n_components=1)
        U = np.array(veca).reshape(-1, 1)
        V = np.array(qb).reshape(-1, 1)
        U_c, V_c = cca.fit_transform(U, V)
        cca_score = np.corrcoef(U_c.T, V_c.T)[0, 1]
        cca_vals.append(np.square(cca_score))
    return(cca_vals)

In [46]:
"""
Try and remove the PCA components and then inverse transform them.
Preserve a certain percentage of the variance and use this to create
new, reduced dimension vectors representing the sentences
"""
def pca_reduce(questions):
    # Fit on the curated questions
    pca = PCA(0.80).fit(questions)
    # Reduce dimensions of curated questions
    quest_comps = pca.transform(questions)
    # Reduce dimensions of the incoming query
    #query_comp = pca.transform(query)
    # Next map them back to the original vector space
    pca_quest = pca.inverse_transform(quest_comps)
    #pca_query = pca.inverse_transform(query_comp)
    filtered = pca.inverse_transform(pca_quest)
    return(pca_quest)

In [116]:
"""
Identify the variance in each group and then reconstruct the vectors
"""
def pca_var(df):
    # Go through each group
    qs=[]
    for group, info in df.groupby('label'):
        # Fit on the curated questions
        pca = PCA(0.90).fit(info['question'].tolist())
        # Reduce dimensions of curated questions
        quest_comps = pca.transform(info['question'].tolist())
        # Next map them back to the original vector space
        pca_quest = pca.inverse_transform(quest_comps)
        for vecqs, txt in zip(pca_quest, info['text'].tolist()):
            qs.append([group, txt, vecqs])
    df_pca = pd.DataFrame(qs, columns=['label', 'txt', 'pcav'])
    return(df_pca)

In [119]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
embed = hub.Module(module_url)

sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2), axis=1)

def get_vectors(session, query, qs_df):
    """Returns the encoded vectors"""
    emba, embb = session.run(
        [sts_encode1, sts_encode2],
        feed_dict={
            sts_input1: query,
            sts_input2: qs_df['text'].tolist()
        })
    return (emba, embb)

INFO:tensorflow:Initialize variable module_4/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_0


INFO:tensorflow:Initialize variable module_4/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_0


INFO:tensorflow:Initialize variable module_4/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_1


INFO:tensorflow:Initialize variable module_4/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_1


INFO:tensorflow:Initialize variable module_4/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_10


...

INFO:tensorflow:Initialize variable module_4/Encoder_en/Transformer/dense/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/dense/kernel


INFO:tensorflow:Initialize variable module_4/Encoder_en/hidden_layers/tanh_layer_0/dense/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/hidden_layers/tanh_layer_0/dense/bias


INFO:tensorflow:Initialize variable module_4/Encoder_en/hidden_layers/tanh_layer_0/dense/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/hidden_layers/tanh_layer_0/dense/bias


INFO:tensorflow:Initialize variable module_4/Encoder_en/hidden_layers/tanh_layer_0/dense/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/hidden_layers/tanh_layer_0/dense/kernel


INFO:tensorflow:Initialize variable module_4/Encoder_en/hidden_layers/tanh_layer_0/dense/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/hidden_layers/tanh_layer_0/dense/kernel


In [ ]:
df1 = get_app_qs("CURATED FILE")

In [123]:
"""
Find the best match using the sentence encodings
"""

results = []
    
queries = [q for q in base_qs['Question']]
pca_df = df1

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    for i, q in enumerate(queries):
        emba, embb = get_vectors(session, [q], pca_df)
        pca_df['question'] = (np.array(embb).tolist())
        pca_qs = pca_var(pca_df)
        cca_scores = get_cca(emba, pca_qs['pcav'].tolist())
        pca_qs['score'] = cca_scores
        sort_df = pca_qs.sort_values('score', ascending=False)
        best = sort_df.head(n=1)
        ans = (best['txt']).tolist()[0]
        results.append([q, ans, (best['label']).tolist()[0],
                        np.round((best['score']).tolist()[0], 3)])
        if i % 10 == 0:
            print(f"Processed {i} queries")
    df = pd.DataFrame(results, columns=['Query', 'Question', 'Label', 'Score'])

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/decomposition/pca.py:423: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)


Processed 0 queries
Processed 10 queries
Processed 20 queries
Processed 30 queries


In [125]:
df.to_csv('use_pca_grp_var_90.csv')

In [ ]:
abs = (pd.read_csv('use_test_rec1.csv')).sort_values('Query', ascending=False)
avg = (pd.read_csv('use_avg_rec1.csv')).sort_values('Query', ascending=False)